In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

import fonctions
from fonctions import scan_xml, modif_chemins_declaration, id_table

# options
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

# dossier
folder = 'D:/Fichiers_dev/test_xml/tables/'
folder2 = 'D:/Fichiers_dev/test_xml/'

data3 = pd.read_csv(folder + 'data_nettoyees.csv')

In [ ]:
# conditions de mapping entre le chemin xml et les champs de la table en cours
mask_champs_ressources = (data3['chemins'].str.contains('participationDirigeantDto/items/', na=False))

champs_ressources2 = data3[mask_champs_ressources]['chemins'].unique()
champs_ressources2

# creation de la table
ressources2 = data3[data3['chemins'].isin(champs_ressources2)][['id_declaration', 'champs', 'valeurs']]
ressources2.head(20)

ressources2.shape

# ajout de l'id_dirigeant
ids_part_dir2 = []
champs_part_dir2 = []

# creation de l'id_dirigeant, qui s'incrémente à chaque nouvelle activité, chaque nouveau id dans le tableau
for element in tqdm(ressources2['champs'].values):
    champs_part_dir2.append(element)
    i = f'd{champs_part_dir2.count("id") - 1}'
    ids_part_dir2.append(i)

# creation de la colonne id_activité dans ressources, en tant que clé étrangère
ressources2['id_activite'] = ids_part_dir2
ressources2.head(20)

# suppression des colonnes qui ne sont pas des ressources
ressources2 = ressources2[ressources2['champs'].isin(['annee', 'montant'])]
ressources2.head()

ressources2.shape

# ajout de l'id_ressources
ids_ressources2 = []
champs_ressources2 = []


# creation de l'id_ressources, qui s'incrémente à chaque nouvelle année
for element in tqdm(ressources2['champs'].values):
    champs_ressources2.append(element)
    i = ressources['id_ressource'].max() + champs_ressources2.count("annee")
    ids_ressources2.append(i)

# creation de la colonne id_ressources dans ressources, en tant que clé primaire
ressources2['id_ressource'] = ids_ressources2
ressources2.head()

# suppression des doublons
ressources2 = ressources2.drop(ressources2[ressources2.duplicated(subset=['id_declaration',
                                                         'champs', 'id_activite', 'id_ressource'], keep=False)].index)

# pivot de la table ressources
ressources2 = ressources2.pivot(index=['id_declaration', 'id_activite', 'id_ressource'],
                                          columns='champs', values='valeurs').reset_index()
ressources2.head()

# encodage id_declaration
ressources2['id_declaration'] = encoder_id_declaration.transform(ressources2['id_declaration'])

# export en csv
ressources2.to_csv(folder + 'ressources2.csv', index=False)